In [2]:

import datetime as dt
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import h2o
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Polygon
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from statsmodels.tools.tools import add_constant

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from h2o.automl import H2OAutoML
from tpot import TPOTClassifier
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
accidents_sub = pd.read_csv("C:\\Users\\reza3\\OneDrive\\Desktop\\AIT\\Machine learning\\group project\\data\\sub_accidents.csv")

In [4]:
accidents_sub.shape

(4016917, 38)

In [8]:
accidents_sub.drop(columns="Unnamed: 0", inplace=True)

In [9]:
accidents_sub.columns

Index(['ID', 'Severity', 'Delay(min)', 'Start_Lat', 'Start_Lng', 'Street',
       'City', 'County', 'State', 'Zipcode', 'Humidity(%)', 'Crossing',
       'Junction', 'Stop', 'Traffic_Signal', 'Sunrise_Sunset',
       'Civil_Twilight', 'Start_Year', 'Start_Month', 'Start_Day',
       'Start_Hour', 'Start_time', 'IsWeekend', 'Temperature(C)',
       'Pressure(cm)', 'Precipitation(cm)', 'Distance(km)', 'Visibility(km)',
       'Wind_Speed(kmph)', 'Delay_ln', 'Severity_new', 'Weather_Bin_Clear',
       'Weather_Bin_Cloudy', 'Weather_Bin_Rainy', 'Weather_Bin_Snowy',
       'Start_Month_December', 'Start_Month_January'],
      dtype='object')

In [ ]:
X = accidents_sub.drop(columns=['ID', 'Severity', 'Delay(min)', 'Street', 'City', 'County', 'State', 'Zipcode',
       'Start_Year', 'Start_Month', 'Start_Day',
       'Start_time','Distance(km)', 'Delay_ln', 'Severity_new')
Y = accidents_sub.loc[:, ['Severity', 'Severity_new', 'Delay(min)', 'Delay_ln', 'Distance(km)']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.05)

assert len(X_train)  == len(Y_train)
assert len(X_test)   == len(Y_test)
X.shape, Y.shape,X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:

'''from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {"n_estimators": [10], 
              "criterion": ["gini"],
              "max_depth": 10}
model = RandomForestClassifier()
model.fit(X_train, Y_train["Severity"])
yhat = model.predict(X_test)

print(classification_report(Y_test["Severity"], yhat))'''

'''grid = GridSearchCV(model, param_grid, refit=True)
grid.fit(X_train, Y_train["Severity"])

print(grid.best_params_)

yhat = grid.predict(X_test["Severity"])

print(classification_report(Y_test["Severity"], yhat))'''
'''from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import  classification_report

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, Y_train["Severity"])
y_pred = ada_clf.predict(X_test)
print("Ada score: ", accuracy_score(Y_test["Severity"], y_pred))
print(classification_report(Y_test["Severity"], y_pred))'''
'''import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
xgb_reg = xgboost.XGBRegressor(early_stopping_rounds=4) 

#not improved after 2 iterations
xgb_reg.fit(X_train, Y_train["Distance(km)"],
                eval_set=[(X_test, Y_test["Distance(km)"])])
y_pred = xgb_reg.predict(X_test)
mse = mean_squared_error(Y_test["Distance(km)"], y_pred)
r2 = r2_score(Y_test["Distance(km)"], y_pred)

print("MSE:", mse)  #notice we are using mse while xgb uses root mse
print("R2:", r2)'''
'''import torch

from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

inputs = torch.from_numpy(X_train.values)
target = torch.from_numpy(Y_train["Delay(min)"].values)
print(inputs.size())
print(target.size())
from torch.utils.data import TensorDataset
# Define dataset
train_ds = TensorDataset(inputs, target)
from torch.utils.data import DataLoader
# Define data loader
batch_size = 500
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
# Build the ANN model

model = Sequential()

# Input layer
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='LeakyReLU'))

# Hidden layers
model.add(Dense(64, activation='LeakyReLU'))
model.add(Dense(48, activation='LeakyReLU'))
model.add(Dense(24, activation='LeakyReLU'))
model.add(Dense(12, activation='LeakyReLU'))

# Output layer
model.add(Dense(1, activation='linear'))  # Linear activation for regression

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, Y_train["Distance(km)"], epochs=1, batch_size=100, validation_data=(X_test, Y_test["Distance(km)"]))
yhat = model.predict(X_test)
r2 = r2_score(Y_test["Distance(km)"], yhat)
weights_and_biases = model.get_weights()
# Evaluate the model
loss = model.evaluate(X_test, Y_test["Distance(km)"])
print(f'Mean Squared Error on Test Set: {loss:.2f}')
print(r2)
'''
from sklearn.cluster import KMeans

#sum of squared distances
ssd = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(X)
    ssd.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), ssd)
plt.xticks(range(2, 20))
plt.grid(True)
plt.ylabel("within cluster variation")
plt.xlabel("k")
plt.title('Elbow curve')

plt.annotate('elbow', xy=(4.3, 220), xytext=(5, 600),  #xytext ---> xy
            arrowprops=dict(arrowstyle="->"))